## Actividad 7 (Generalización)

**Script que genera una tabla que contiene el último meger y el último mayor a 1.3(level merger)**  

In [1]:
import merger_tree_tools as mtt
import numpy as np
from astropy.table import Table
from astropy.cosmology import Planck15 as cosmo
import h5py
import pandas as pd

In [2]:
path_lin ='/home/ramiro/Facultad/Tesis/Datos/Actividad 7/' 
# path_win =

In [3]:
table  = pd.read_csv(path_lin+'Aperturas_RefL0100N1504_snap_28_aperture_30.dat')

In [4]:
table_a = table[(table['Stars_Mass']>=1e10)].copy()
table_a

,GalaxyID,GroupID,SnapNum,Redshift,SubGroupNumber,CentreOfPotential_x,CentreOfPotential_y,CentreOfPotential_z,Stars_Mass,SF_Mass,...,NSF_Hydrogen,NSF_Oxygen,NSF_Metallicity,StarFormationRate,InitialMassWeightedStellarAge,ApertureSize,Mass_Star,SFR_30pkpc,Group_R_Crit200,Group_M_Crit200
0,9498012,28000000000002,28,2.220446e-16,6,8.913222,35.828056,54.839039,1.538708e+11,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,10.139413,30,1.307842e+11,0.000000,1411.312012,3.002119e+14
1,9508202,28000000000002,28,2.220446e-16,7,6.904262,35.705917,57.397373,7.604099e+10,2.281596e+09,...,0.727174,0.004796,0.011342,0.841832,9.455478,30,6.641890e+10,0.823222,1411.312012,3.002119e+14
2,9512113,28000000000002,28,2.220446e-16,8,8.682913,35.504921,54.217369,1.345128e+11,1.514760e+10,...,0.696796,0.010628,0.024796,12.094972,7.656161,30,1.183275e+11,10.984530,1411.312012,3.002119e+14
3,9517736,28000000000002,28,2.220446e-16,9,8.278317,37.063808,54.914932,2.232997e+10,2.754609e+09,...,0.714222,0.007443,0.017283,1.334875,8.117089,30,2.024596e+10,1.322288,1411.312012,3.002119e+14
4,9519365,28000000000002,28,2.220446e-16,10,8.262937,36.454063,55.901993,4.833353e+10,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,8.520104,30,3.944613e+10,0.000000,1411.312012,3.002119e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13131,10365607,28000000002202,28,2.220446e-16,0,35.516003,10.797990,34.600616,1.226640e+10,1.538676e+09,...,0.740806,0.002266,0.005160,0.456897,7.618259,30,1.191011e+10,0.433110,179.046371,6.129222e+11
13145,10859653,28000000002570,28,2.220446e-16,0,61.355499,48.616516,65.236801,1.094640e+10,3.207207e+09,...,0.741599,0.002183,0.004858,1.192947,6.309848,30,1.063636e+10,1.192947,168.661011,5.124048e+11
13155,11104654,28000000002905,28,2.220446e-16,0,97.197350,82.709518,43.676567,1.200869e+10,1.247963e+09,...,0.743705,0.001579,0.003758,0.506203,6.814752,30,1.192049e+10,0.506203,159.761032,4.354551e+11
13164,11247394,28000000003127,28,2.220446e-16,0,88.764885,47.226864,81.905136,1.186733e+10,2.167418e+09,...,0.737761,0.002874,0.006602,0.686905,6.787498,30,1.124763e+10,0.604550,152.119400,3.759434e+11


In [5]:
data = {'GalaxyID_test':[],'GroupID_test':[],'snap_lm':[],'z_lm':[],'L_lm':[],'snap_lmm':[],'z_lmm':[],'L_lmm':[]}

DATA_merge = pd.DataFrame(data=data)

In [6]:
simu='RefL0100N1504'
snap=28

In [7]:
long = len(table_a['GalaxyID'])

# long = 17

# Usuario y contraseña para conectarse a EAGLE DataBase
usr='cht015'
pwd='BH457tfj'

# Descargar merger tree completo de la galaxia deseada
# Nombre y alias de la tabla de la cual se quieren descargar datos
table='SubHalo'
table_alias='sub'

# Variables que se quiere descargar. OJO!! Asegurarse que estas variables
# estén en la tabla deseada.
columns=[
          'GalaxyID','LastProgID','TopLeafID','DescendantID',
          'SnapNum','Redshift','Stars_Mass','SF_Mass','GroupID'
        ]

#Loop para todas las galaxias de la tabla
for p in np.arange(long):

    galid = table_a['GalaxyID'].to_numpy()[p]
 
    # Descargar todos los IDS necesarios de la galaxia deseada

    myIDs    = mtt.retrieve_ids(usr,pwd,simu,snap,galid)

    raw_tree = mtt.download_merger_tree(usr,pwd,simu,myIDs['GalaxyID'],myIDs['LastProgID'],
                                  table=table,table_alias=table_alias,columns=columns)
    
    # Aplicar condiciones a las galaxias del árbol, si es necesario
    mask = (
            (raw_tree['Stars_Mass']>0e6)&(raw_tree['SF_Mass']>0)
           )

    # Armar arbol sólo con galaxias seleccionadas según condiciones anteriores
    tree = {}
    for key in raw_tree.keys():
        tree[key]=raw_tree[key][mask]
        
    # Ordeno las galaxias según SnapNum creciente
    mask_order = (tree['SnapNum']).argsort()
    for key in tree.keys():
        tree[key] = tree[key][mask_order]

    # Armo un diccionario con solo la main branch del árbol
    main_branch = {}
    # Select galaxies in the main branch 
    mask_main = np.logical_and(tree['GalaxyID']>=myIDs['GalaxyID'],
                               tree['GalaxyID']<=myIDs['TopLeafID']) 

    for key in tree.keys():
        main_branch[key] = tree[key][mask_main]

        
    xplot_main = main_branch['SnapNum']
    
    # Calculo level of merger
    level_merger=[1]       # Inicializo con un 1, porque la primer galaxia del main branch 
                           # no viene de ninguna fusión...
    
    for k in range(np.size(xplot_main)-1):
        m1=main_branch['Stars_Mass'][k]+main_branch['SF_Mass'][k]
        mask=(tree['DescendantID']==main_branch['GalaxyID'][k+1]) & (tree['SnapNum']!=28)
        # La condición de snapnum distinto a 28 es porque galaxias a z=0 no tienen descendiente,
        # y se les asigna como DescendantID su propio GalaxyID
        m2=np.sum(tree['Stars_Mass'][mask])+np.sum(tree['SF_Mass'][mask])
        level=m2/m1
        level_merger=np.append(level_merger,level)  

    i=1
    j=1
    if level_merger[-i]>1:
        snap_lm = main_branch['SnapNum'][-1]
        z_lm    = main_branch['Redshift'][-1]
        L_lm    = level_merger[-1]
    else:
        while level_merger[-i]<=1:
            if i<len(level_merger):
                i+=1
                snap_lm = main_branch['SnapNum'][-i]
                z_lm    = main_branch['Redshift'][-i]
                L_lm    = level_merger[-i]
            
            else:
                snap_lm = np.nan
                z_lm    = np.nan
                L_lm    = np.nan
                break
                
    if level_merger[-j]>1.3:
        snap_lm = main_branch['SnapNum'][-1]
        z_lm    = main_branch['Redshift'][-1]
        L_lm    = level_merger[-1]
    else:
        while level_merger[-j]<=1.3:
            if j<len(level_merger):
                j+=1
                snap_lmm= main_branch['SnapNum'][-j]
                z_lmm   = main_branch['Redshift'][-j]
                L_lmm   = level_merger[-j]
            else:
                snap_lmm= np.nan
                z_lmm   = np.nan
                L_lmm   = np.nan
                break

    data = {'GalaxyID_test':table_a['GalaxyID'].to_numpy()[p],'GroupID_test':table_a['GroupID'].to_numpy()[p],
            'snap_lm':snap_lm,'z_lm':z_lm,'L_lm':L_lm,'snap_lmm':snap_lmm,'z_lmm':z_lmm,'L_lmm':L_lmm}

    DATA_merge = DATA_merge.append(data,ignore_index=True)
    
    print('Porcentaje:',round(float((p+1)/long),2),end='\r')


In [8]:
DATA_merge = DATA_merge.astype({'GalaxyID_test':int,'GroupID_test':int,'snap_lm':float,
                                'z_lm':float,'L_lm':float,'snap_lmm':float,'z_lmm':float,
                                'L_lmm':float})
DATA_merge

,GalaxyID_test,GroupID_test,snap_lm,z_lm,L_lm,snap_lmm,z_lmm,L_lmm
0,9498012,28000000000002,22.0,6.151898e-01,1.369978,22.0,0.615190,1.369978
1,9508202,28000000000002,26.0,1.827099e-01,1.000422,19.0,1.004122,1.325826
2,9512113,28000000000002,28.0,2.220446e-16,1.002306,24.0,0.365669,1.545269
3,9517736,28000000000002,28.0,2.220446e-16,1.147535,NaN,NaN,NaN
4,9519365,28000000000002,25.0,2.709011e-01,1.008288,5.0,7.049566,3.614620
...,...,...,...,...,...,...,...,...
3633,10365607,28000000002202,28.0,2.220446e-16,1.007574,11.0,3.527977,1.465920
3634,10859653,28000000002570,28.0,2.220446e-16,1.004723,12.0,3.016505,1.357690
3635,11104654,28000000002905,25.0,2.709011e-01,1.007583,NaN,NaN,NaN
3636,11247394,28000000003127,28.0,2.220446e-16,1.008742,NaN,NaN,NaN


**Nos guardamos la tabla**

In [9]:
# DATA_merge.to_csv(path_or_buf= path_lin+'/Actividad_7(data_level_merge).dat',index=False)   # Nos guardamos los datos